# Immorthon

In [1]:
# install kagglehub
!pip install kagglehub[pandas-datasets]

# imports
import kagglehub
from kagglehub import KaggleDatasetAdapter

import pandas as pd
pd.set_option('display.max_colwidth', None)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 1.2 MB/s eta 0:00:001.2 MB/s eta 0:00:01m


In [2]:
dataSetName = "kartmaan/dictionnaire-francais"
filePath = "dico.csv"

df = kagglehub.dataset_load(
    adapter=KaggleDatasetAdapter.PANDAS,
    handle=dataSetName,
    path=filePath,
)

print("dataset saved at /content/sample_data/dico.csv")

100%|██████████| 11.2M/11.2M [00:18<00:00, 626kB/s] 

Extracting zip of dico.csv...


dataset saved at /content/sample_data/dico.csv


In [ ]:
# print column names
print(df.columns)

# get df[Mot="car"]["Définitions"]
foo = df[df["Mot"] == "Toulouse"]["Définitions"]

# print a short description of the dataset
print(df[df["Mot"] == "Python"])

Index(['Mot', 'Définitions'], dtype='object')
           Mot  \
17891   Python   
470975  Python   

                                                                                                                                                                                                   Définitions  
17891   ['Genre de serpents, les plus grands des ophidiens, ainsi nommés par allusion au Python de la fable, serpent monstrueux qui fut tué par Apollon près de Delphes.', 'Langage informatique interprété.']  
470975                                                                          ['Animal monstrueux de Delphes dans la mythologie grecque.', 'Langage de programmation inventé par Guido van Rossum en 1990.']  
